In [46]:
import requests
import string

from lxml import etree
from lxml import html

In [51]:

queryParam = {'fromDate': '2015-12-20'}
proxies = {'http': 'proxy.wellsfargo.com:8080', 'https': 'proxy.wellsfargo.com:8080'}
parser = etree.HTMLParser()
r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', data=queryParam, proxies=proxies, timeout=5.0)

# htmltext = unicode(string.replace(r.text, '&nbsp;', ' '))


# root = etree.fromstring(htmltext)
r.headers

{'content-length': '15780', 'content-language': 'en-US', 'content-encoding': 'gzip', 'age': '1', 'x-cdn': 'BYPASS-BYPASS-0-0-1', 'vary': 'Accept-Encoding,User-Agent', 'server': 'G2 server', 'last-modified': 'Tue, 22 Dec 2015 08:19:06 GMT', 'connection': 'Keep-Alive', 'etag': 'W/"15780-1450772345082"', 'cache-control': 'max-age=60, must-revalidate', 'date': 'Tue, 22 Dec 2015 08:19:21 GMT', 'x-powered-by': 'Servlet/3.0', 'content-type': 'text/html; charset=UTF-8'}

In [49]:
tree = etree.fromstring(r.content, parser=parser)
#htmltext
trList =tree.xpath('//table[@id=\'list\']/tr[not(@mr)]')
for tr in trList:
    for td in tr.iterchildren():
        print(td.text)

20:05
HX 327
呼和浩特
wmocrk,香港航空
B
到閘口 00:16 (22/12/2015)
00:05
CX 636
新加坡
wmocpa,國泰航空公司
B
到閘口 00:01
00:05
UO 689
大阪/關西
wmohke,香港快運航空
B
到閘口 23:56 (21/12/2015)
00:15
BX 391,OZ 9741
釜山
wmoabl,釜山航空,wmoaar,韓亞航空
B
到閘口 00:16
00:25
7C 2107
首爾/仁川
wmojja,濟州航空
B
到閘口 01:03
00:35
MM 067
大阪/關西
wmoapj,樂桃航空
B
到閘口 00:19
00:40
HX 231,FM 8231,MU 8982
上海/浦東
wmocrk,香港航空,wmocsh,上海航空公司,wmoces,中國東方航空
B
到閘口 00:44
00:40
KA 975,CX 5975,CA 6523
北京
wmohda,港龍航空公司,wmocpa,國泰航空公司,wmocca,中國國際航空公司
B
到閘口 01:23
00:40
KE 615
釜山
wmokal,大韓航空
B
到閘口 00:37
01:00
JW 345
東京/成田
wmovnl,香草航空
B
到閘口 01:08
01:20
UO 1863
大阪/關西
wmohke,香港快運航空
B
到閘口 01:32
01:45
UO 1621
首爾/仁川
wmohke,香港快運航空
B
到閘口 01:52
04:00
UO 623
東京/羽田
wmohke,香港快運航空
B
到閘口 03:50
05:00
CX 138,BA 4134
悉尼
wmocpa,國泰航空公司,wmobaw,英國航空公司
B
到閘口 04:26
05:00
CX 746
巴林,杜拜
wmocpa,國泰航空公司
B
到閘口 04:39
05:05
CX 714,AA 8901
新加坡
wmocpa,國泰航空公司,wmoaal,美國航空
B
到閘口 06:17
05:25
KA 251
仰光
wmohda,港龍航空公司
B
到閘口 05:14
05:25
KA 397,CX 5397
東京/羽田
wmohda,港龍航空公司,wmocpa,國泰航空公司
B
到閘口 04:48
05:30
HX 762,EY 6168
